In [1]:
# %% 
import numpy as np
from sklearn.model_selection import train_test_split
#%%
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


print(tf.__version__)

2.16.1


In [2]:

def grad_loss(v_gt, v):
    # Gradient loss
    loss = tf.reduce_mean(tf.abs(v - v_gt), axis=[1,2,3])
    jy = v[:,:,1:,:,:] - v[:,:,:-1,:,:]
    jx = v[:,:,:,1:,:] - v[:,:,:,:-1,:]
    jy_ = v_gt[:,:,1:,:,:] - v_gt[:,:,:-1,:,:]
    jx_ = v_gt[:,:,:,1:,:] - v_gt[:,:,:,:-1,:]
    loss += tf.reduce_mean(tf.abs(jy - jy_), axis=[1,2,3])
    loss += tf.reduce_mean(tf.abs(jx - jx_), axis=[1,2,3])
    return loss

In [3]:
def uNet(input, time, lat, lon, height, kernel = [5, 3, 3], nodes = [72, 144, 288, 576]):
    '''
    This function defines a U-Net architecture
    :param input: the main-input layer
    :param time: the time input layer
    :param lat, lon, height: additional fields
    :param kernel: Kernel-sizes (default = [5, 3, 3])
    :param nodes: different neuron-sizes if needed (default = [72, 144, 288, 576])
    :return: last layer of constructed model
    '''

    # set Timesteps
    TS = 3
    ##################################################### 1st Block ####################################################
    conv1 = Conv3D(filters      = nodes[0],
                   kernel_size  = (TS, kernel[0], kernel[0]),
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(input)
    mergetime = Concatenate(axis=4)([conv1, lat, lon, height])
    conv1 = Conv3D(filters      = nodes[0],
                   kernel_size  = kernel[0],
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(mergetime)

    pool1 = MaxPooling3D(pool_size = (1, 2, 2))(conv1)

    ##################################################### 2nd Block ####################################################
    conv2 = Conv3D(filters      = nodes[1],
                   kernel_size  = (TS, kernel[1], kernel[1]),
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(pool1)
    conv2 = Conv3D(filters      = nodes[1],
                   kernel_size  = (TS, kernel[1], kernel[1]),
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(conv2)

    pool2 = MaxPooling3D(pool_size = (1, 2, 2))(conv2)

    ##################################################### 3rd Block ####################################################
    conv3 = Conv3D(filters      = nodes[2],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(pool2)
    conv3 = Conv3D(filters      = nodes[2],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation='relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(conv3)

    pool3 = MaxPooling3D(pool_size = (1, 2, 2))(conv3)

    ##################################################### 4th Block ####################################################
    conv4 = Conv3D(filters      = nodes[3],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation='relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(pool3)
    conv4 = Conv3D(filters      = nodes[3],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation='relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(conv4)

    ####################################################### TIME #######################################################
    # Merge time-layer at this point
    mergetime = Concatenate(axis=4)([conv4, time])

    ################################################### UP 3rd Block ###################################################
    # Up-Size again
    up3   = UpSampling3D(size = (1, 2, 2))(mergetime)
    up3   = Conv3D(filters              = nodes[2],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   kernel_initializer   = 'he_normal')(up3)

    # Skip connection
    merge3 = Concatenate(axis=4)([conv3, up3])

    conv3 = Conv3D(filters              = nodes[2],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(merge3)
    conv3 = Conv3D(filters              = nodes[2],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(conv3)

    ################################################### UP 2nd Block ###################################################
    up2 = UpSampling3D(size = (1, 2, 2))(conv3)
    up2 = Conv3D(filters                = nodes[1],
                 kernel_size            = (TS, kernel[1], kernel[1]),
                 activation             = 'relu',
                 padding                = 'same',
                 kernel_initializer     = 'he_normal')(up2)

    # Skip connection
    merge2 = Concatenate(axis=4)([conv2, up2])

    conv2 = Conv3D(filters              = nodes[1],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(merge2)
    conv2 = Conv3D(filters              = nodes[1],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(conv2)

    ################################################### UP 1st Block ###################################################
    up1 = UpSampling3D(size = (1, 2, 2))(conv2)
    up1 = Conv3D(filters                = nodes[0],
                 kernel_size            = (TS, kernel[0], kernel[0]),
                 activation             = 'relu',
                 padding                = 'same',
                 kernel_initializer     = 'he_normal')(up1)

    merge1 = Concatenate(axis=4)([conv1, up1])

    conv1 = Conv3D(filters              = nodes[0],
                   kernel_size          = (TS, kernel[0], kernel[0]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(merge1)
    conv1 = Conv3D(filters              = nodes[0],
                   kernel_size          = (TS, kernel[0], kernel[0]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(conv1)

    # last layer is the output
    output = conv1

    return output


In [4]:
def get_model(PS=32, loss = grad_loss, optimizer = 'adam', nodes = [72, 144, 288, 576], residual = False):
    '''
    This function creates the DCN-architecture (residual = False) or RPN-architecture (residual = True).
    :param PS: Patch size
    :param loss: loss function (default = grad_loss)
    :param optimizer: optimizer (default = 'adam')
    :param nodes: different neuron-sizes if needed (default = [72, 144, 288, 576])
    :param residual: boolean toggeling between RPN (True) and DCN (False)
    :return: Model
    '''

    # Input layers
    main_input  = Input(shape = (3, PS, PS, 1))
    time        = Input(shape = (3, int(PS/8), int(PS/8), 1))
    lat         = Input(shape = (3, PS, PS, 1))
    lon         = Input(shape = (3, PS, PS, 1)) 
    height      = Input(shape = (3, PS, PS, 1))

    # Load U-Net
    unet        = uNet(main_input, time, lat, lon, height, nodes = nodes)

    # Define output layer after U-Net
    temp_out    = Conv3D(filters        = 1,
                         kernel_size    = (3, 1, 1),
                         activation     = 'linear',
                         padding        = 'same',
                         data_format    = "channels_last")(unet)

    def transpose_fn(x):
      return tf.transpose(x, perm=[0, 4, 2, 3, 1])
    temp_out = Lambda(transpose_fn)(temp_out)
    # # Define output layer after U-Net
    temp_out    = Conv3D(filters        = 2,
                         kernel_size    = (1, 1, 1),
                         activation     = 'linear',
                         padding        = 'valid',
                         data_format    = "channels_last")(temp_out)
    temp_out = Lambda(transpose_fn)(temp_out)
    
    # residual layer
    if residual:
        temp_out = Add()([main_input[:,1,:,:], temp_out])

    # create model with the defined Layers
    model       = Model(inputs          = [main_input, time, lat, lon, height],
                        outputs         = temp_out)

    # compile with defined loss and optimizer
    model.compile(loss      = loss,
                  optimizer = optimizer,
                  metrics   = ['mse', 'mae', 'mape'])

    return model

def main():
  model = get_model() # DCN
#   model = get_model(residual=True) # RPN
  model.summary()
  print(model.summary())

We want data in the size of patch_size x patch_size in time step of 3

In [5]:
# Load data
wind_speed_data = np.load('tensor.npy')  # Assuming wind speed is in the first channel
print(wind_speed_data.shape)
wind_speed_data = wind_speed_data[:, :, :, 0]  # Extract wind speed channel

# Normalize the data
wind_speed_data = (wind_speed_data - np.min(wind_speed_data)) / (np.max(wind_speed_data) - np.min(wind_speed_data))

(52704, 23, 34, 2)


In [6]:
wind_speed_data.shape

(52704, 23, 34)

In [7]:
import numpy as np
import tensorflow as tf

# Reshape data to fit the model input shape
def reshape_data(data, target_shape=(32, 32)):
    # Reshape data to the target shape
    data = data[..., np.newaxis]
    resized_data = tf.image.resize(data, target_shape, method='bilinear')
    data = resized_data.numpy()
    return data

def create_patches(data, time_steps):
    num_samples = data.shape[0] - time_steps + 1
    training_patches = []
    patches = []

    

    for i in range(num_samples):
        patch = data[i:i+time_steps]
        patches.append(patch[1::2])
        training_patches.append(patch[::2])
    training_timestep = time_steps//2+1
    relative_time = np.zeros((time_steps//2+1, data.shape[1]//8, data.shape[2]//8, 1))
    # relative_time = np.zeros((3, 4, 4, 1))

    # Assign specific values to each slice along the first axis
    # relative_time[0, :, :, 0] = -1
    # relative_time[1, :, :, 0] = 0
    # relative_time[2, :, :, 0] = 1
    for t in range(training_timestep):
        relative_time[t, :, :, 0] = t*2
    relative_time_patches = np.tile(relative_time, (num_samples, 1, 1, 1, 1))
    training_patches = np.array(training_patches)
    patches = np.array(patches)

    return training_patches, relative_time_patches, patches


# Parameters
patch_size = 32
time_steps_label = 5

# Reshape data
reshaped_data = reshape_data(wind_speed_data, target_shape=(32, 32))

# Create label patches with time_steps_label
input_patches, input_time_patches, label_patches = create_patches(reshaped_data, time_steps_label)

# input_patches, input_time_patches = input_patches[:label_patches.shape[0],:,:,:,:], input_time_patches[:label_patches.shape[0],:,:,:,:]

print("Input patches shape:", input_patches.shape)
print("Input time patches shape:", input_time_patches.shape)
print("Label patches shape:", label_patches.shape)



Input patches shape: (52700, 3, 32, 32, 1)
Input time patches shape: (52700, 3, 4, 4, 1)
Label patches shape: (52700, 2, 32, 32, 1)


In [2]:
# get one month
one_month_size = input_patches.shape[0]//12
input_patches = input_patches[:one_month_size]
input_time_patches = input_time_patches[:one_month_size]
label_patches = label_patches[:one_month_size]

NameError: name 'input_patches' is not defined

In [23]:
# # Split data
# train_inputs, val_inputs, train_inputs_time, val_inputs_time = train_test_split(
#     input_patches, input_time_patches, test_size=0.2, random_state=42)

# train_labels, val_labels, train_labels_time, val_labels_time = train_test_split(
#     label_patches, test_size=0.2, random_state=42)
# print("Training input patches shape:", train_inputs.shape)
# print("Training input time patches shape:", train_inputs_time.shape)
# print("Val input time patches shape:", val_inputs_time.shape)
# print("Validation input patches shape:", val_inputs.shape)
# print("Training label patches shape:", train_labels.shape)
# print("Validation label patches shape:", val_labels.shape)

In [24]:
# Define the size of the training set
train_size = int(0.85 * len(input_patches))  # 80% for training

# Create indices
indices = np.arange(len(input_patches))
np.random.seed(42)  # Set seed for reproducibility
np.random.shuffle(indices)

# Split indices into training and validation
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Use the indices to split the data
train_inputs = input_patches[train_indices]
val_inputs = input_patches[val_indices]

train_inputs_time = input_time_patches[train_indices]
val_inputs_time = input_time_patches[val_indices]

train_labels = label_patches[train_indices]
val_labels = label_patches[val_indices]


In [25]:
print("Training input patches shape:", train_inputs.shape)
print("Training input time patches shape:", train_inputs_time.shape)
print("Val input time patches shape:", val_inputs_time.shape)
print("Validation input patches shape:", val_inputs.shape)
print("Training label patches shape:", train_labels.shape)
print("Validation label patches shape:", val_labels.shape)

Training input patches shape: (3732, 3, 32, 32, 1)
Training input time patches shape: (3732, 3, 4, 4, 1)
Val input time patches shape: (659, 3, 4, 4, 1)
Validation input patches shape: (659, 3, 32, 32, 1)
Training label patches shape: (3732, 2, 32, 32, 1)
Validation label patches shape: (659, 2, 32, 32, 1)


In [26]:
val_inputs = val_inputs[:10,:,:,:,:]

val_inputs_time = val_inputs_time[:10,:,:,:,:]

val_labels = val_labels[:10,:,:,:,:]

In [27]:
train_labels.shape

(3732, 2, 32, 32, 1)

In [29]:
from keras.callbacks import ModelCheckpoint

# Define the model
epochs = 50
batch_size = 32
model = get_model(PS=batch_size)

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',  # Filepath where the model will be saved
    monitor='val_loss',        # Monitor the validation loss
    save_best_only=True,       # Save only the best model
    save_weights_only=False,   # Save the whole model, not just weights
    mode='min',                # Mode 'min' means it will save the model with the minimum validation loss
    verbose=1                  # Verbosity mode
)

# Train the model with validation and checkpointing
history = model.fit(
    [train_inputs, train_inputs_time, train_inputs, train_inputs, train_inputs],
    train_labels,
    validation_data=([val_inputs, val_inputs_time, val_inputs, val_inputs, val_inputs], val_labels),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[checkpoint_callback],
    verbose=1
)


Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - loss: 1.8453 - mae: 1.6264 - mape: 600.1868 - mse: 81.8075 
Epoch 1: val_loss improved from inf to 0.01509, saving model to best_model.keras
117/117 ━━━━━━━━━━━━━━━━━━━━ 3047s 26s/step - loss: 1.8335 - mae: 1.6159 - mape: 596.3328 - mse: 81.2744 - val_loss: 0.0151 - val_mae: 0.0122 - val_mape: 3.5255 - val_mse: 2.4802e-04
Epoch 2/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - loss: 0.0168 - mae: 0.0143 - mape: 4.5482 - mse: 3.5405e-04 
Epoch 2: val_loss improved from 0.01509 to 0.00660, saving model to best_model.keras
117/117 ━━━━━━━━━━━━━━━━━━━━ 2084s 18s/step - loss: 0.0168 - mae: 0.0142 - mape: 4.5429 - mse: 3.5332e-04 - val_loss: 0.0066 - val_mae: 0.0042 - val_mape: 1.5884 - val_mse: 3.4209e-05
Epoch 3/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - loss: 0.0212 - mae: 0.0186 - mape: 5.7420 - mse: 7.0684e-04 
Epoch 3: val_loss did not improve from 0.00660
117/117 ━━━━━━━━━━━━━━━━━━━━ 2053s 18s/step - loss: 0.0212 - mae: 0.0186 - m

KeyboardInterrupt: 

In [8]:
test_size = 100

In [9]:
test_inputs = input_patches[-test_size:,:,:,:,:]

test_inputs_time = input_time_patches[-test_size:,:,:,:,:]

test_labels = label_patches[-test_size:,:,:,:,:]

In [10]:
def transpose_fn(x):
      return tf.transpose(x, perm=[0, 4, 2, 3, 1])

In [11]:
from keras.models import load_model

# Load the saved model
best_model = load_model('best_model.keras',custom_objects={'grad_loss': grad_loss, "transpose_fn": transpose_fn})

# Now you can use `best_model` to make predictions or further evaluations
predictions = best_model.predict([test_inputs,test_inputs_time,test_inputs,test_inputs,test_inputs])

4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step


In [ ]:
# from keras.models import load_model

# # Load the saved model
# best_model = load_model('best_model.keras',custom_objects={'grad_loss': grad_loss, "transpose_fn": transpose_fn})

# # Now you can use `best_model` to make predictions or further evaluations
# predictions = best_model.predict([val_inputs,val_inputs_time,val_inputs,val_inputs,val_inputs])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [12]:
np.sum((test_labels[:,:,:,:,:] - predictions[:,:,:,:,:])/ test_labels[:,:,:,:,:])

-807.3833

In [13]:
test_labels.shape

(100, 2, 32, 32, 1)

In [14]:
# mean interpolation
interpolation = [(test_inputs[:,1:2,:,:,:] + test_inputs[:,0:1,:,:,:]) / 2, (test_inputs[:,2:3,:,:,:] + test_inputs[:,1:2,:,:,:]) / 2]
interpolation_result = np.concatenate(interpolation, axis=1).shape

In [15]:
np.mean(np.abs((test_labels - predictions)/ test_labels))

0.011145579

In [16]:
np.mean(np.abs((test_labels - interpolation) / test_labels))

0.020976929

In [18]:
from scipy.ndimage import zoom
def trilinear_downscale(inputs, target_size):
    """
    Downscales the second dimension (dimension 1) using trilinear interpolation.
    
    Args:
        inputs (np.ndarray): Input array of shape (N, D, H, W, C)
        target_size (int): The target size for the second dimension (D)
    
    Returns:
        np.ndarray: Downscaled array of shape (N, new_D, H, W, C)
    """
    # Compute the zoom factor for downscaling
    zoom_factor = target_size / inputs.shape[1]
    
    # Create a zoom array where the zoom factor is applied to dimension 1
    zoom_factors = [1.0, zoom_factor, 1.0, 1.0, 1.0]
    
    # Apply the zoom function
    downscaled = zoom(inputs, zoom_factors, order=1)  # order=1 for trilinear interpolation
    
    return downscaled
target_size = 2  # New size for the dimension 1

downscaled_interpolation = trilinear_downscale(test_inputs, target_size)
print(downscaled_interpolation.shape)

(100, 2, 32, 32, 1)


In [19]:
mse_prediction = np.mean((test_labels - predictions) ** 2)
mse_baseline = np.mean((test_labels - interpolation) ** 2)
mse_interpolation = np.mean((test_labels - downscaled_interpolation) ** 2)
print(mse_prediction, mse_baseline, mse_interpolation)

1.238095e-05 6.520077e-05 3.6313635e-05
